# Image Segmentation - 7150, 7128

We intend to perform image segmentation. Image segmentation means that we can group similar pixels together and give these grouped pixels the same label. 

The grouping problem is a clustering problem. We want to study the use of K-means on the Berkeley Segmentation Benchmark. 

In [1]:
# import necessary libraries

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

## 1. Download the Dataset and Understand the Format (5 points)

## 2. Visualize the image and the G.T Segmentation (5 points)

In [ ]:
for dirname, _, filenames in os.walk('/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 3. Segmentation using K-means (15 points)